# ***Installation Requirements***



In [ ]:
!pip install tensorflow==2.12.0

# ***Mount Google Drive***

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/ieee_tnsm25_tes-models/TES-Transformer/Capacity_Forecasting

# ***Imports***

In [ ]:
import os
import math
import time
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import random
import numpy as np
from torch.utils.data import DataLoader
from data_loading import create_dataset, Dataset
from config import get_config
from loss_modules import *

# ***TES-Transformer***

In [ ]:
# CONFIGURATION SETTINGS

# List of the services to be tested
services = ['Facebook', 'Instagram', 'Snapchat']

# Number of clusters types
num_clusterss = [1]

# List of alphas to be tested
alphas = [1, 2, 3, 5]

# Define the number of training epochs
epochs = 20

# Define the number of training batch size
batch_size = 288

# Define the number of train, validation and test samples
train_samples = 16128
val_samples = 4032
test_samples = 2016

# Define the input window and output window of the prediction
input_window = 6
output_window = 1
block_len = input_window + output_window

# Define simulation run seed
num_run = 0
torch.manual_seed(0)

# Define the device type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# ACTOR-CRITIC IMPORTS AND SETTINGS
from sac import SAC
from Utils.buffer import ReplayBuffer
from torch.autograd import Variable

# Define state, action size and agents
state_size = 2
action_size = 20
possible_tau = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
num_agents = 1

# Define general parameters
gamma = 0.99
tau_ac = 0.001
rollout_threads = 1
ac_batch_size = 10
pol_hidden_dim = 512
critic_hidden_dim = 512
pi_lr = 0.01
q_lr = 0.01
norm_rews = True

# Define Actor-Critic iterations and experience-buffer length
ac_iterations = 50
buffer_length = ac_iterations

In [ ]:
# TES-Transformer implementation

# Implementation of the Positional Encoding module
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = 1 / (10000 ** ((2 * np.arange(d_model)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term[0::2])
        pe[:, 1::2] = torch.cos(position * div_term[1::2])
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]


# Implementation of the Transformer Attention module
class TransAm(nn.Module):
    def __init__(self, maximum, feature_size=10, num_layers=1, dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        self.input_embedding = nn.Linear(6, feature_size)
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(feature_size, 1)
        self.init_weights()
        self.maximum = maximum
        self.logistic = nn.Sigmoid()

        #Set initial parameter of the Exponential Smoothing formula and make it learnable
        init_lev_sms = []
        init_lev_sms.append(nn.Parameter(torch.Tensor([0.5]), requires_grad=True))
        self.init_lev_sms = nn.ParameterList(init_lev_sms)

    def init_weights(self):
        initrange = 0.1
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if src.dim() == 3 and src.size(-1) == 1:
            src = src.squeeze(-1)

        src = self.input_embedding(src)
        src = src.unsqueeze(0)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output.squeeze(0)


# Implementation of the module for Input/Output sequences
def create_inout_sequences(input_data, input_window ,output_window):
    inout_seq = []
    L = len(input_data)
    block_num =  L - block_len + 1

    for i in range( block_num ):
        train_seq = input_data[i : i + input_window]
        train_label = input_data[i + output_window : i + input_window + output_window]
        inout_seq.append((train_seq ,train_label))

    return torch.FloatTensor(np.array(inout_seq))


# Implementation of the module for data retrieval
def get_data(epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_window, output_window, service, run):

    config = get_config('Traffic', epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_window, output_window)
    if num_clusters > 1:
        data = '../../../Dataset/' + service + '/time_load_cor_matrix_' + str(num_clusters) + '_clust.npy'
        train, val, test = create_dataset(data, config['chop_train'], config['chop_val'], config['chop_test'], clustering=True, cluster=run)
    else:
        data = '../../../Dataset/' + service + '/time_load_cor_matrix.npy'
        train, val, test = create_dataset(data, config['chop_train'], config['chop_val'], config['chop_test'])

    train_data = train[0]
    val_data = val[0]
    test_data = test[0]
    max_train = np.max(train[0])
    max_val = np.max(val[0])
    mean_val = np.mean(val[0])
    std_val = np.std(val[0])
    max_test = np.max(test[0])

    train_sequence = create_inout_sequences(train_data, input_window, output_window)
    val_sequence = create_inout_sequences(val_data, input_window, output_window)
    test_sequence = create_inout_sequences(test_data, input_window, output_window)

    return train_sequence.to(device), val_sequence.to(device), test_sequence.to(device), max_train, max_val, mean_val, std_val, max_test


# Implementation of the module for batch data retrieval
def get_batch(model, input_data, i , batch_size, tau):

    input_data = input_data + 0.1
    batch_len = min(batch_size, len(input_data) - i)
    data = input_data[ i:i + batch_len ]
    lev_sms = model.logistic(torch.stack([model.init_lev_sms[idx] for idx in [0]]).squeeze(1))

    # Calculate initial levels
    levels = []
    max_levels = []
    init_lev = input_data[0, 0, 0]
    levels.append(init_lev)

    # Calculate level for the current timestep
    for i in range(1, data.shape[0]):
        new_lev = lev_sms * (data[i,0,0]) + (1 - lev_sms) * levels[i - 1]
        levels.append(new_lev)

    for i in range(data.shape[0], data.shape[0] + input_window):
        new_lev = lev_sms * (data[data.shape[0] - 1, 0, i - data.shape[0]]) + (1 - lev_sms) * levels[i - 1]
        levels.append(new_lev)

    levels = torch.tensor(levels)

    for i in range(data.shape[0]):

        data[i,0,:] = torch.div(data[i,0,:],levels[i + input_window])
        data[i,1,:] = torch.div(data[i,1,:],levels[i + input_window])
        max_window = torch.max(data[i,0,:])*tau
        if max_window>1e-3:
            data[i,0,:] = torch.div(data[i,0,:], max_window)
            data[i,1,:] = torch.div(data[i,1,:], max_window)
            max_levels.append(max_window)
        else:
            data[i,0,:] = torch.div(data[i,0,:], 1)
            data[i,1,:] = torch.div(data[i,1,:], 1)
            max_levels.append(1)
    max_levels = torch.tensor(max_levels)
    input = torch.stack([item[0] for item in data]).view((batch_len, input_window))
    target = torch.stack([item[1][-1]for item in data]).view((batch_len, 1))

    return input, target, levels, max_levels


# Implementation of the module for training
def train(model, optimizer, criterion, train_data, tau):
    model.train()
    total_loss = 0.

    for batch, i in enumerate(range(0, len(train_data), batch_size)):
        data, targets, levels, max_levels = get_batch(model, train_data, i , batch_size, tau)
        one_window = np.array(data.cpu())[0]
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval


# Implementation of the module for validation and testing
def evaluate(eval_model, criterion, data_source, epoch, tau):
    eval_model.eval()
    total_loss = 0.
    eval_result = torch.Tensor(0)
    truth = torch.Tensor(0)
    with torch.no_grad():
        numpy_levels = []
        numpy_max_levels = []
        for i in range(len(data_source)):
            data, target, levels, max_levels = get_batch(eval_model, data_source, i , 1, tau)
            output = eval_model(data)
            total_loss += criterion(output, target).item()
            eval_result = torch.cat((eval_result, output[-1].view(-1).cpu()), 0)
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            numpy_levels.append(levels[-1].cpu())
            numpy_max_levels.append(max_levels[-1].cpu())


    final_levels = np.array(numpy_levels)
    final_max_levels = np.array(numpy_max_levels)

    eval_result = eval_result * final_max_levels * final_levels
    eval_result = eval_result - 0.1
    truth = truth * final_levels * final_max_levels
    truth = truth - 0.1

    return total_loss / i, eval_result, truth, final_levels, final_max_levels

In [ ]:
# SIMULATION RUNS

# Iterate over the services
for service in services:

    # Iterate over the number of antennas
    for num_clusters in num_clusterss:

        # Iterate over the alphas
        for alpha in alphas:

            # Iterate for the number of clusters
            for run in range(num_clusters):

                # Obtain the data of the simulation
                train_data, val_data, test_data, max_train, max_val, avg_val, std_val, max_test = get_data(epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_window, output_window, service, run)

                # Set the maximum
                maximum = max_train

                # Single model per configuration case
                max_avg_val = avg_val
                max_std_val = std_val

                # Turn on exploration
                exploration = True

                # Actor Critic initialization
                ac_model = SAC.init_from_env(state_size, action_size, num_agents, gamma, tau_ac, pi_lr=pi_lr, q_lr=q_lr, pol_hidden_dim=pol_hidden_dim, critic_hidden_dim=critic_hidden_dim)
                replay_buffer = ReplayBuffer(buffer_length, num_agents, [(state_size) for j in range(num_agents)], [action_size for k in range(num_agents)])

                # Prepare the model
                ac_model.prep_rollouts(device='cpu')

                for ac_iter in range(ac_iterations):

                    # Check whether turn off exploration
                    if ac_iter >= (ac_iterations - 5) :
                        exploration = False

                    # Get the average value in the validation dataset (normalized state of the Actor Critic)
                    avg_validation_traffic = avg_val / max_avg_val

                    # Get the standard deviation value in the validation dataset (normalized state of the Actor Critic)
                    std_validation_traffic = std_val / max_std_val


                    # Determine the state, action and next state for the Actor Critic
                    complete_agent_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                    agent_state = np.ndarray(shape=(state_size,))
                    agent_state[0] = avg_validation_traffic
                    agent_state[1] = std_validation_traffic
                    for k in range(num_agents):
                        complete_agent_state[0,k] = agent_state
                    torch_state = [Variable(torch.Tensor(np.vstack(complete_agent_state[:, j])), requires_grad=False) for j in range(num_agents)]


                    # Get the action from the Actor Critic
                    torch_action = ac_model.step(torch_state, explore=exploration)
                    agent_actions = [ac.data.numpy() for ac in torch_action]
                    tau = possible_tau[np.argmax(agent_actions[0])]

                    # Model initialization
                    run_id = service + '/Alpha_' + str(alpha) + '/Simulation_' + str(num_run)
                    model = TransAm(max_train).to(device)

                    # Select loss function and optimizer and scheduler and learning rate
                    criterion = AlphaLoss(alpha, 1, "cuda")
                    lr = 0.005
                    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
                    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

                    # Run model trainer
                    for epoch in range(epochs):
                        train(model, optimizer, criterion, train_data, tau)
                        scheduler.step()

                    # Run model validator
                    loss, predictions, actuals, final_levels, final_max_levels = evaluate(model, criterion, val_data, epochs, tau)

                    # Find the peak
                    peak = torch.max(actuals)

                    # Move to numpy arrays
                    predictions = predictions.cpu().numpy()
                    actuals = actuals.cpu().numpy()
                    peak = peak.cpu().numpy()

                    # Find the val_loss
                    val_loss, over, sla = evaluate_costs_single_clust(predictions, actuals, peak, alpha)



                    # Determine the next state for the Actor Critic
                    complete_agent_post_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                    agent_post_state = np.ndarray(shape=(state_size,))
                    agent_post_state[0] = avg_validation_traffic
                    agent_post_state[1] = std_validation_traffic
                    for k in range(num_agents):
                        complete_agent_post_state[0,k] = agent_post_state
                    torch_post_state = [Variable(torch.Tensor(np.vstack(complete_agent_post_state[:, j])), requires_grad=False) for j in range(num_agents)]
                    rewards = np.ndarray(shape=(rollout_threads, num_agents))
                    rewards[0,0] = -val_loss


                    # Save the experience in the replay buffer
                    replay_buffer.push(complete_agent_state, agent_actions, rewards, complete_agent_post_state)


		            # Perform training of the Radio agent model
                    if len(replay_buffer) >= (ac_batch_size) and ac_iter < (ac_iterations - 5):
                        ac_model.prep_training(device='cpu')
                        sample = replay_buffer.sample(ac_batch_size, to_gpu=False, norm_rews=norm_rews)
                        ac_model.update_critic(sample)
                        ac_model.update_policies(sample)
                        ac_model.update_all_targets()
                        ac_model.prep_rollouts(device='cpu')

                        # Perform a sample inference
                        complete_agent_state = np.ndarray(shape=(rollout_threads, num_agents), dtype=object)
                        agent_state = np.ndarray(shape=(state_size,))
                        agent_state[0] = avg_validation_traffic
                        agent_state[1] = std_validation_traffic
                        for k in range(num_agents):
                            complete_agent_state[0,k] = agent_state
                        torch_state = [Variable(torch.Tensor(np.vstack(complete_agent_state[:, j])), requires_grad=False) for j in range(num_agents)]
                        torch_action = ac_model.step(torch_state, explore=False)
                        agent_actions = [ac.data.numpy() for ac in torch_action]
                        test_tau = possible_tau[np.argmax(agent_actions[0])]
                        os.makedirs('Results/' + run_id, exist_ok=True)
                        np.save('Results/' + run_id + '/test_tau_ac_iter_' + str(ac_iter) + '.npy', tau)


                # Save the model
                file_path = os.path.join('AC_Models', run_id)
                model_path = os.path.join(file_path, 'ac_model_sim_' + str(num_run))
                os.makedirs(file_path, exist_ok=True)
                ac_model.save('AC_Models/' + run_id + '/ac_model_sim_' + str(num_run))



                # Run the optimized model after taus optimization
                train_data, val_data, test_data, max_train, max_val, avg_val, std_val, max_test = get_data(epochs, num_clusters, batch_size, train_samples, val_samples, test_samples, alpha, input_window, output_window, service, run)

                # Model initialization
                run_id = service + '/Alpha_' + str(alpha) + '/Simulation_' + str(num_run)
                model = TransAm(max_train).to(device)

                # Select loss function and optimizer and scheduler and learning rate
                criterion = AlphaLoss(alpha, 1, "cuda")
                lr = 0.005
                optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
                scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.95)

                # Run model trainer
                for epoch in range(epochs):
                    train(model, optimizer, criterion, train_data, tau)
                    scheduler.step()

                # Test the model
                loss, predictions, actuals, final_levels, final_max_levels = evaluate(model, criterion, test_data, epochs, tau)

                # Find the peak
                peak = torch.max(actuals)

                # Move to numpy arrays
                predictions = predictions.cpu().numpy()
                actuals = actuals.cpu().numpy()
                peak = peak.cpu().numpy()

                # Find the different parts of alphaloss
                den_loss, over, sla = evaluate_costs_single_clust(predictions, actuals, peak, alpha)
                sla_cost = den_loss - over
                print("Denormalized Alpha-loss: ", den_loss)


                # Store the results
                os.makedirs('Results/' + run_id, exist_ok=True)
                np.save('Results/' + run_id + '/tes-transformer_predictions_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), predictions)
                np.save('Results/' + run_id + '/tes-transformer_actuals_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), actuals)
                np.save('Results/' + run_id + '/tes-transformer_alpha_loss_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), den_loss)
                np.save('Results/' + run_id + '/tes-transformer_over_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), over)
                np.save('Results/' + run_id + '/tes-transformer_sla_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), sla_cost)
                np.save('Results/' + run_id + '/tes-transformer_tau_%s_%d_%d_%d_%d.npy'%(service, num_clusters, run, num_run, alpha), tau)
